In [ ]:
from scatter.main import save, make_callable
from pprint import pprint


def sample(i: int, j: float = 4.0) -> list[float]:
    return i + j


pprint(save(sample).dict())



In [ ]:
import redis

r = redis.Redis()
r.flushall()